In [1]:
#Importing libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import math

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

#Removing long warning message texts.
import warnings
warnings.filterwarnings('ignore')

In [2]:
#Importing data
df = sns.load_dataset('tips')

In [3]:
# df = sns.load_dataset('tips')
df.head(3)

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3


# Data cleaning and preparation
Plan:
* Data summary and shape.
* Check and deal with mising values.
* Check for and deal with duplicates.
*  Check and remove ourtliers.

In [4]:
#Data types
print(df.info())
df.dtypes

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244 entries, 0 to 243
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   total_bill  244 non-null    float64 
 1   tip         244 non-null    float64 
 2   sex         244 non-null    category
 3   smoker      244 non-null    category
 4   day         244 non-null    category
 5   time        244 non-null    category
 6   size        244 non-null    int64   
dtypes: category(4), float64(2), int64(1)
memory usage: 7.4 KB
None


total_bill     float64
tip            float64
sex           category
smoker        category
day           category
time          category
size             int64
dtype: object

The data set has 244 entries, with 7 variables well labeled; 3 numerical variables and 4 categorical variables. We can see that there are no missing values in the data.

In [5]:
#Check for and deal with duplicates
dups = df.duplicated()
dups.value_counts()

False    243
True       1
Name: count, dtype: int64

In [6]:
#Removing the one duplicate found
df_cleaned = df.drop_duplicates()

#change the ncolumn name sex to gender
df_cleaned = df_cleaned.rename(columns={'sex': 'gender'})

One duplicate was found in the dataset and removed, leaving behind 243 entries. Due to the non existence of unique identifying variables, no further search for duplicates basing on particular columns will be done.

***Now all the data cleaning and EDA was done in EDA notebook this was forked from, so i am going straight for the regression.***

# Regression 

For only the numerical variables will be used.

The EDA above included the outliers, however when removed (I did that in the EDA versions). And although outliers are representative of occurances in tipping where there are generous client from time to time, they will skew out model.

In [7]:
#Lazy outlier removal
df_ready = df_cleaned[(df['tip'] <= 7)]
df_ready

,total_bill,tip,gender,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4
...,...,...,...,...,...,...,...
239,29.03,5.92,Male,No,Sat,Dinner,3
240,27.18,2.00,Female,Yes,Sat,Dinner,2
241,22.67,2.00,Male,Yes,Sat,Dinner,2
242,17.82,1.75,Male,No,Sat,Dinner,2


***Splitting dependent and independent.***

In [8]:
#first creating tip ratio (tips/total bill)
df_ready['tip_ratio'] = df_ready['tip']/ df_ready['total_bill']

In [9]:
#Selecting x and y.
X = df_ready[['total_bill', 'size', 'tip_ratio']]
y = df_ready['tip']

# Data seperation
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [10]:
# Fit the model on the training data
model = LinearRegression()
model.fit(x_train, y_train)

LinearRegression()

In [11]:
#Preditions on the x_test data
y_pred = model.predict(x_test)

# Evaluate the model's performance
mse = mean_squared_error(y_test, y_pred)
print(f'MSE: {mse:.2f}')
print(f'Root mean square error: {math.sqrt(mse):.2f}')

MSE: 0.38
Root mean square error: 0.61


A root mean Square error(√0.38) of $0.616 is rather high and the model may need to be flexed down.

However, after several iterations such as removing the tip ratio, or running it alone, the results show that run with all three is best.

***Although, I came to the realization that the tip ratio is computed from an already available tip***

⊳⊳⊳⊳ To solve this, I suggest computing a constant tip ratio as ***average(tip_ratio)***.